In [ ]:
import json
import pennylane as qml
import pennylane.numpy as np
from typing import List, Dict, Tuple
import pandas as pd
import time
import psutil
import os
from datetime import datetime
import logging

# Quantum Base-10 Controller Class
class QuantumBase10Controller:
    def __init__(self, num_qubits=4):
        self.num_qubits = num_qubits
        self.device = qml.device("default.qubit", wires=num_qubits)

    def base10_model(self, alpha: float) -> np.ndarray:
        x = alpha / 10

        @qml.qnode(self.device)
        def circuit():
            # Base-10 state preparation with heaviside transitions
            for i in range(self.num_qubits):
                theta = np.pi * np.heaviside(np.sin(10 * (2**i) * np.pi * x + np.pi/4), 0.5)
                qml.RY(theta, wires=i)

            # Enhanced entanglement for base-10 encoding
            for i in range(self.num_qubits - 1):
                qml.CNOT(wires=[i, i+1])
                qml.PhaseShift(np.pi/10, wires=i)

            return qml.probs(wires=range(self.num_qubits))

        # Execute the quantum node
        return circuit()

    def generate_base10_coefficients(self):
        """Generate coefficients for perfect base-10 state preparation"""
        coefs = np.linspace(0, 10, 10, endpoint=False)
        return [c + np.pi/40 for c in coefs]

    def test_base10_states(self):
        coefs = self.generate_base10_coefficients()
        results = []

        for alpha in coefs:
            probs = self.base10_model(alpha)
            state_index = np.argmax(probs)

            results.append({
                'alpha': alpha,
                'state_index': state_index,
                'probability': probs[state_index],
                'probabilities': probs.tolist()
            })

            print(f"Alpha: {alpha:.3f}, State: {state_index}, Probability: {probs[state_index]:.4f}")

        return results


# Main Analysis Class
class QubitScalingAnalyzer:
    def __init__(self, start_qubits=4, max_qubits=16, step=2):
        self.start_qubits = start_qubits
        self.max_qubits = max_qubits
        self.step = step
        self.results_dir = f"scaling_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        os.makedirs(self.results_dir, exist_ok=True)

        # Set up logging
        logging.basicConfig(
            filename=f"{self.results_dir}/analysis.log",
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )

    def analyze_state(self, probs: np.ndarray, num_qubits: int) -> Dict:
        try:
            state_index = np.argmax(probs)
            state_binary = format(state_index, f'0{num_qubits}b')

            # Safe entropy calculation
            probs = np.array(probs)
            valid_probs = probs[probs > 1e-10]
            entropy = -np.sum(valid_probs * np.log2(valid_probs + 1e-10))

            return {
                'dominant_state': state_binary,
                'max_probability': float(probs[state_index]),
                'entropy': float(entropy),
                'num_significant_states': int(np.sum(probs > 0.01)),
                'hilbert_space_dimension': 2**num_qubits
            }
        except Exception as e:
            logging.error(f"State analysis failed: {str(e)}")
            raise

    def run_single_qubit_test(self, num_qubits: int) -> Dict:
        try:
            controller = QuantumBase10Controller(num_qubits)

            # Test points
            alphas = controller.generate_base10_coefficients()
            results = []

            start_time = time.time()
            start_memory = psutil.Process().memory_info().rss / 1024 / 1024

            for alpha in alphas:
                probs = controller.base10_model(alpha)
                analysis = self.analyze_state(probs, num_qubits)
                results.append(analysis)

            end_time = time.time()
            end_memory = psutil.Process().memory_info().rss / 1024 / 1024

            return {
                'num_qubits': num_qubits,
                'execution_time': end_time - start_time,
                'memory_usage_mb': end_memory - start_memory,
                'avg_entropy': np.mean([r['entropy'] for r in results]),
                'max_probability': max([r['max_probability'] for r in results]),
                'avg_significant_states': np.mean([r['num_significant_states'] for r in results]),
                'results': results
            }

        except Exception as e:
            logging.error(f"Test failed for {num_qubits} qubits: {str(e)}")
            raise

    def run_analysis(self):
        summary_data = []

        for num_qubits in range(self.start_qubits, self.max_qubits + 1, self.step):
            print(f"\nAnalyzing {num_qubits} qubits...")
            logging.info(f"Starting analysis for {num_qubits} qubits")

            try:
                result = self.run_single_qubit_test(num_qubits)
                summary_data.append(result)

                # Save detailed results
                df = pd.DataFrame(result['results'])
                df.to_csv(f"{self.results_dir}/qubits_{num_qubits}_results.csv", index=False)

                logging.info(f"Completed analysis for {num_qubits} qubits")

            except Exception as e:
                print(f"Failed at {num_qubits} qubits: {str(e)}")
                logging.error(f"Analysis failed at {num_qubits} qubits: {str(e)}")
                break

        self.generate_summary_report(summary_data)

    def generate_summary_report(self, summary_data):
        if not summary_data:
            print("No data collected!")
            return

        df = pd.DataFrame([{k:v for k,v in d.items() if k != 'results'} for d in summary_data])

        # Save summary data
        df.to_csv(f"{self.results_dir}/scaling_summary.csv", index=False)

        analysis_text = [
            "=== Quantum System Scaling Analysis ===\n",
            f"Analysis completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
            f"\nSuccessfully tested qubit ranges: {df['num_qubits'].min()} to {df['num_qubits'].max()}",
            f"\nPerformance Metrics:",
            f"- Average execution time per qubit: {df['execution_time'].mean():.2f} seconds",
            f"- Average memory usage per qubit: {df['memory_usage_mb'].mean():.2f} MB",
            f"\nQuantum State Metrics:",
            f"- Average entropy range: {df['avg_entropy'].min():.2f} to {df['avg_entropy'].max():.2f}",
            f"- Maximum achieved probability: {df['max_probability'].max():.4f}",
            f"\nDetailed scaling analysis saved in: {self.results_dir}"
        ]

        report = '\n'.join(analysis_text)
        print(report)

        with open(f"{self.results_dir}/scaling_analysis_report.txt", 'w') as f:
            f.write(report)
            f.write("\n\nFull Data Summary:\n")
            f.write(df.to_string())

# Run the analysis
def main():
    analyzer = QubitScalingAnalyzer(start_qubits=4, max_qubits=16, step=2)
    analyzer.run_analysis()

if __name__ == "__main__":
    main()
